# **Entrada**

In [2]:
import pandas as pd


def datos(data):
    
    file =  pd.read_csv(data + ".csv", sep = ";", header =0)
    
    file = file[["estu_consecutivo.1","punt_lenguaje","punt_matematicas","punt_biologia","punt_quimica",
                   "punt_fisica","punt_ciencias_sociales","punt_filosofia","punt_ingles","exito"
                   ]]
    
    file["exito"] = file["exito"].replace({1: True, 0: False})
    
    
    return file

def datosEntrenamiento(data):
    
    file =  pd.read_csv(data + ".csv", sep = ";", header =0)
    
    file = file[["punt_lenguaje","punt_matematicas","punt_biologia","punt_quimica",
                   "punt_fisica","punt_ciencias_sociales","punt_filosofia","punt_ingles","exito"
                   ]]
    
    file["exito"] = file["exito"].replace({1: True, 0: False})
    
    
    return file

In [20]:
from google.colab import files
uploades = files.upload()

Saving TEST 4.csv to TEST 4.csv


# **Arbol**

In [3]:
class Nodo:
  def __init__(self,pregunta,entropia,probabilidad,decision=None):
      self.pregunta = pregunta
      self.entropia = entropia
      self.probabilidad  = probabilidad
      self.decision = decision
      self.izquierda = None
      self.derecha = None
class Arbol:
    def __init__(self):
        self.raiz= None

    def retornarRaiz(self):
      return self.raiz   

    def agregar(self,val,root,deci,decision1=None):
        if deci==None:
            self.raiz = Nodo(val[0],val[1],val[2],decision1)
            return  self.raiz
        else:
          if deci ==True:
            root.derecha = Nodo(val[0],val[1],val[2],decision1)
            return  root.derecha
          elif deci==False:
            root.izquierda = Nodo(val[0],val[1],val[2],decision1)
            return  root.izquierda
    

# **Metodos Auxiliares**

In [4]:
class Question:
    
    def __init__(self,column,value):
        if column == "":
          self.column = column
          self.value  = value
          self.imprimir =   "Hoja"
        else:
          self.column = column
          self.value  = value
          self.imprimir =   "Es %s >= %s?" % (self.column, str(self.value)) 
    def match(self,example):
        if is_numeric(example):
            return example >= self.value
        else:
            return example == self.value
    
    def __repr__(self):
        condition ="=="
        if is_numeric(self.value):
            condition=">="
        return "Es %s %s %s?" % (self.column, condition, str(self.value))
     

def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

In [5]:
def partition(index,value,dataset):
    question = Question(index,value)
    filtro = (dataset[index]>=value)
    right = pd.DataFrame(dataset.loc[filtro])

    filtro1 = (dataset[index]<value)
    left = pd.DataFrame(dataset.loc[filtro1])
    
    left = left.drop([index],axis=1)
    right = right.drop([index],axis=1)

    right = right.reset_index(drop = True)
    left = left.reset_index(drop = True)
    
    return left, right

In [6]:
import math

In [7]:
def targetcalculate(file):
  file = file["exito"]
  size = file.size
  true, false =0,0
  filtro = (file==True)
  trueData = pd.DataFrame(file.loc[filtro])
  true = trueData[trueData.columns[0]].size

  filtro1 = (file==False)
  falseData = pd.DataFrame(file.loc[filtro1])
  false = falseData[falseData.columns[0]].size
  
  partTrue, partFalse = 0,0
  if true!=0:
    partTrue = -(true/size)*math.log2((true/size))
  if false!=0:
    partFalse = (false/size)*math.log2((false/size))
  target = partTrue-partFalse
  return round(target,2)


In [8]:
def entropia(file,column,cuestion,target):
  falseData,trueData = partition(column,cuestion,file)
  sizef = falseData[falseData.columns[0]].size
  sizet = trueData[trueData.columns[0]].size
  size = file[file.columns[0]].size
  entropy = target-((sizef/size*targetcalculate(falseData))+(sizet/size*targetcalculate(trueData)))
  return round(entropy,2)

In [9]:
def mejorDiv(file,target):
    bestEntropy, bestsplit,bestcolumn = 0,0,""
    for i in range(0,file.columns.size-1):#iterator aumenta a partir de 1
      column = file[file.columns[i]]
      if is_numeric(column[0]):
        entropy = entropia(file,file.columns[i],file.mean().iloc[i],target)
        if entropy>bestEntropy:
          bestEntropy= entropy
          bestsplit = round(file.mean().iloc[i],2)
          bestcolumn = file.columns[i]
    return bestEntropy,bestsplit,bestcolumn

In [10]:

import numpy as np
def seleccionarDatos(archivo):
  p_train = 0.80
  archivo['EsEntrenamiento'] = np.random.uniform(0, 1, len(archivo)) <= p_train
  entrenamiento, prueba = archivo[archivo['EsEntrenamiento']==True], archivo[archivo['EsEntrenamiento']==False]
  

  entrenamientoFiltro = (archivo['EsEntrenamiento']==True) 
  datosEntrenamiento = pd.DataFrame(archivo.loc[entrenamientoFiltro])
  datosEntrenamiento = datosEntrenamiento.reset_index(drop = True)
  datosEntrenamiento = datosEntrenamiento.iloc[:,1:-1]

  pruebaFiltro = (archivo['EsEntrenamiento']==False) 
  datosPrueba = pd.DataFrame(archivo.loc[pruebaFiltro])
  datosPrueba = datosPrueba.reset_index(drop = True)
  datosPrueba = datosPrueba.iloc[:,:-1]

  return datosEntrenamiento,datosPrueba

In [11]:
class Prediccion:
  def __init__(self):
    self.true = 0
    self.false = 0

  def prediccion(self,datos,datos_y,raiz,iterador):
    if raiz.decision != None:
      if raiz.decision == True and datos_y["exito"].iloc[iterador]==True and datos["estu_consecutivo.1"].iloc[iterador]==datos_y["estu_consecutivo.1"].iloc[iterador]:
        self.true += 1
        return 
      elif raiz.decision ==False and datos_y["exito"].iloc[iterador]==False and datos["estu_consecutivo.1"].iloc[iterador]==datos_y["estu_consecutivo.1"].iloc[iterador]:
        self.false += 1
        return 
    elif raiz.pregunta.match(datos[raiz.pregunta.column].iloc[iterador]) == True:
      self.prediccion(datos,datos_y,raiz.derecha,iterador)
    else:
      self.prediccion(datos,datos_y,raiz.izquierda,iterador)
  
  def retornarTrue(self):
    return self.true
  
  def retornarFalse(self):
    return self.false

In [12]:
def predicciones(prueba,y_prueba):
    predic = Prediccion()
    tamaño = prueba[prueba.columns[0]].size
    for i in range(0,tamaño):
      predic.prediccion(prueba,y_prueba,tree.retornarRaiz(),i)
    return predic.retornarTrue(), predic.retornarFalse()
  

In [13]:
tree = Arbol()


In [14]:
def calcularMejorDiv(profundidad,dot,archivo,target,iterador,padre=None,probabil =None,valor=None):
    x = mejorDiv(archivo,target)
    if profundidad==1 and padre==None:
      padre2 = tree.agregar([Question(x[2],x[1]),x[0],probabil],padre,valor,archivo["exito"].describe().top)
      cadena =  "\" RAIZ \\n probabilidad ="+str(probabil)+"\\n Decision ="+str(archivo["exito"].describe().top)+"\";\n"
      dot.write(cadena)
      if archivo["exito"].describe().top == True:
        cadena2 = "\" RAIZ \\n probabilidad ="+str(probabil)+"\\n Decision ="+str(archivo["exito"].describe().top)+"\"[style= filled,fillcolor=green];\n"
      else: 
        cadena2 = "\" RAIZ \\n probabilidad ="+str(probabil)+"\\n Decision ="+str(archivo["exito"].describe().top)+"\"[style= filled,fillcolor=red];\n"
      dot.write(cadena2)
      return
    elif x[2]=="" or iterador == profundidad or target==0:
      padre2 = tree.agregar([Question(x[2],x[1]),x[0],probabil],padre,valor,archivo["exito"].describe().top)
      cadena = "\""+padre.pregunta.imprimir+" \\n probabilidad ="+str(padre.probabilidad)+"\" -> \" Hoja \\n probabilidad ="+str(probabil)+"\\n Decision ="+str(archivo["exito"].describe().top)+" \";\n"
      dot.write(cadena)
      if archivo["exito"].describe().top == True:
        cadena2 = "\" Hoja \\n probabilidad ="+str(probabil)+"\\n Decision ="+str(archivo["exito"].describe().top)+" \" [style= filled,fillcolor=green];\n"
      else: 
        cadena2 = "\" Hoja \\n probabilidad ="+str(probabil)+"\\n Decision ="+str(archivo["exito"].describe().top)+" \" [style= filled,fillcolor=red];\n"
      dot.write(cadena2)
      return
    else:
      padre2 = tree.agregar([Question(x[2],x[1]),x[0],probabil],padre,valor)
      if padre!=None:
        cadena = "\""+padre.pregunta.imprimir+" \\n probabilidad ="+str(padre.probabilidad)+"\" -> \""+ Question(x[2],x[1]).imprimir+" \\n probabilidad ="+str(probabil)+"\";\n"
        dot.write(cadena)
      false ,true = partition(x[2],x[1],archivo)
      filtro = (false['exito']==True)
      proba = pd.DataFrame(false.loc[filtro])
      proba = proba[proba.columns[0]].size
      probabilidad = (proba*100)/false[false.columns[0]].size

      filtro1 = (true['exito']==True)
      proba1 = pd.DataFrame(true.loc[filtro1])
      proba1 = proba1[proba1.columns[0]].size
      probabilidad1 = (proba1*100)/true[true.columns[0]].size

      targetFalse = targetcalculate(false)
      calcularMejorDiv(profundidad,dot,false,targetFalse,iterador+1,padre2,round(probabilidad,2),False)
      
      targetTrue = targetcalculate(true)
      calcularMejorDiv(profundidad,dot,true,targetTrue,iterador+1,padre2,round(probabilidad1,2),True)

In [15]:
def crearMatriz(predicVerdad,predicFals,verdaderosTamanio,falsosTamanio):
    TP, TN, FP, FN = 0,0,0,0
    if predicVerdad<verdaderosTamanio and predicFals<falsosTamanio:
      TP = predicVerdad
      TN = predicFals
      FP = verdaderosTamanio - predicVerdad
      FN = falsosTamanio - predicFals
    elif predicVerdad>verdaderosTamanio and predicFals<falsosTamanio:
      TP = verdaderosTamanio
      TN = predicFals
      FN = falsosTamanio - predicFals
    elif predicVerdad<verdaderosTamanio and predicFals>falsosTamanio:
      TP = predicVerdad
      TN = falsosTamanio
      FP = verdaderosTamanio - predicVerdad
    elif  predicVerdad==verdaderosTamanio and predicFals==falsosTamanio:
      TP = predicVerdad
      TN = predicFals
    matrizDeConfusion = np.array([[TP,TN],[FP,FN]])
    exactitud = ((TP +TN) / (TP + TN + FP + FN))*100
    return exactitud,matrizDeConfusion


# **Principal**

In [16]:
def correr(nombreArchivo,profundidad):
  entrada = datos(nombreArchivo)
  entrenamiento,x_prueba = seleccionarDatos(entrada)
  y_prueba = x_prueba[["estu_consecutivo.1","exito"]]
  target = targetcalculate(entrenamiento)
  if profundidad!=1:
    archi1=open("Arbol.txt","w") 
    archi1.write("digraph arbol{\n")
    calcularMejorDiv(profundidad,archi1,entrenamiento,target,1)
    archi1.write("}\n")
    predicVerdad,predicFals = predicciones(x_prueba,y_prueba)

    filtro = (x_prueba['exito']==True)
    verdaderos = pd.DataFrame(x_prueba.loc[filtro])
    verdaderosTamanio = verdaderos[verdaderos.columns[0]].size

    filtro1 = (x_prueba['exito']==False)
    falsos = pd.DataFrame(x_prueba.loc[filtro1])
    falsosTamanio = falsos[falsos.columns[0]].size
    exac,matrizDeConfusion = crearMatriz(predicVerdad,predicFals,verdaderosTamanio,falsosTamanio)
    print("Exactitud",exac,"%")
    print(matrizDeConfusion)
  else:
    archi1=open("Arbol.txt","w") 
    archi1.write("digraph arbol{\n")
    calcularMejorDiv(profundidad,archi1,entrenamiento,target,1)
    archi1.write("}\n")
    print("Como solo se genero la raiz, entonces no hay matriz de confusion y no se asigna exactitud")

In [17]:
def correr2(archivoTrain,archivoTest,profundidad):
  entrenamiento = datosEntrenamiento(archivoTrain)
  x_prueba = datos(archivoTest)
  y_prueba = x_prueba[["estu_consecutivo.1","exito"]]
  target = targetcalculate(entrenamiento)
  if profundidad !=1:
    archi1=open("Arbol2.txt","w") 
    archi1.write("digraph arbol{\n")
    calcularMejorDiv(profundidad,archi1,entrenamiento,target,1)
    archi1.write("}\n")

    predicVerdad,predicFals = predicciones(x_prueba,y_prueba)

    filtro = (x_prueba['exito']==True)
    verdaderos = pd.DataFrame(x_prueba.loc[filtro])
    verdaderosTamanio = verdaderos[verdaderos.columns[0]].size

    filtro1 = (x_prueba['exito']==False)
    falsos = pd.DataFrame(x_prueba.loc[filtro1])
    falsosTamanio = falsos[falsos.columns[0]].size
    exac,matrizDeConfusion = crearMatriz(predicVerdad,predicFals,verdaderosTamanio,falsosTamanio)
    print("Exactitud",exac,"%")
    print(matrizDeConfusion)
  else:
    archi1=open("Arbol2.txt","w") 
    archi1.write("digraph arbol{\n")
    calcularMejorDiv(profundidad,archi1,entrenamiento,target,1)
    archi1.write("}\n")
    print("Como solo se genero la raiz, entonces no hay matriz de confusion y no se asigna exactitud")

In [19]:
correr("TRAIN 4",3)

Exactitud 71.96232339089482 %
[[ 942 1350]
 [ 676  217]]


In [ ]:
correr2("TRAIN 4","TEST 4",3)

Exactitud 73.16 %
[[13486 19436]
 [ 9032  3046]]


In [ ]:
tree.retornarRaiz().pregunta

Es punt_ingles >= 52.34?

#**Pruebas**


In [ ]:
#f,t = partition("fami_numlibros","26 A 100 LIBROS",archivo)# asi se hace unaparticion

In [ ]:
a = archivo[archivo.columns[0]].size #calcula el tamaño de filas de un dataframe

In [ ]:
x = range(0,archivo.columns.size-2)#rango de las columnas

In [ ]:
y = archivo.mean().iloc[0]#calcula el promedio de una columna denotada por el iloc